In [1]:
"""
Main pipeline for ARC task solving using execution-based similarity.

Pipeline: Program Similarity → Pattern Discovery → Code Generation
"""

import re
import json
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass
from pathlib import Path
import os
import sys

# print("Script starting...", flush=True)
# sys.stdout.flush()

from src.vlm_prompter import VLMPrompter
from src.vlm_client import VLMConfig, create_client, BaseVLMClient
from src.utils.library import ProgramLibrary, calculate_grid_similarity
from src.utils.dsl import *
from src.utils.constants import *
# from utils.render_legacy import grid_to_base64_png_oai_content

print("Imports done...", flush=True)
sys.stdout.flush()

/home/flowers/work/llms_ftw/src/vlm_prompter.py:359: SyntaxWarning: invalid escape sequence '\ '
  diagonal_mirror(grid) -> Grid/Patch                  # diagonal \ mirror


Imports done...


In [2]:
@dataclass
class TaskResult:
    """Result of attempting to solve a task"""
    task_id: str
    success: bool
    score: float
    program: Optional[str] = None
    phase1_output: Optional[str] = None
    error: Optional[str] = None


def test_program(program_code: str, task: Dict) -> Tuple[float, List[Tuple[Any, Any, bool]]]:
    """
    Test a program against task training examples.
    
    Returns:
        - Average score across examples
        - List of (expected_output, actual_output, passed) tuples
    """
    namespace = globals().copy()
    
    try:
        exec(program_code, namespace)
        
        if 'solve' not in namespace:
            return 0.0, []
        
        solve_fn = namespace['solve']
        scores = []
        results = []
        
        for example in task['train']:
            inp = example['input']
            expected = example['output']
            if isinstance(inp, list):
                inp = tuple(tuple(row) for row in inp)
            try:
                actual = solve_fn(inp)
                score = calculate_grid_similarity(actual, expected)
                scores.append(score)
                results.append((expected, actual, score == 1.0))
            except Exception as e:
                scores.append(0.0)
                results.append((expected, None, False))
        
        avg_score = sum(scores) / len(scores) if scores else 0.0
        return avg_score, results
        
    except Exception as e:
        return 0.0, []


def extract_code_from_response(response: str) -> Optional[str]:
    """Extract Python code from LLM response."""
    python_blocks = re.findall(r'```python\n(.*?)```', response, re.DOTALL)
    
    if python_blocks:
        for block in python_blocks:
            if 'def solve' in block:
                return block.strip()
        return python_blocks[0].strip()
    
    match = re.search(r'(def solve\(I\):.*?)(?=\n\ndef|\n\nif __name__|$)', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    return None


def solve_task(
    task: Dict,
    task_id: str,
    vlm_client_phase1: BaseVLMClient,
    vlm_client_phase2: BaseVLMClient,
    prompter: VLMPrompter,
    library: ProgramLibrary,
    verbose: bool = True,
    n_workers: int = None,
    timeout: int = 2,
    log_dir: str = "logs"
) -> TaskResult:
    """
    Solve a single ARC task using execution-based pipeline.
    
    Pipeline:
    1. Find similar programs by execution (parallelized)
    2. Phase 1: Pattern discovery (natural language) with similar programs
    3. Phase 2: Code generation with pattern + similar programs
    
    Args:
        task: Task dictionary with 'train' examples
        task_id: Unique task identifier
        vlm_client: VLM client for queries
        prompter: Prompt builder
        library: Program library
        verbose: Print progress
        n_workers: Number of parallel workers (None = auto)
        timeout: Timeout per program execution in seconds
        log_dir: Directory to save logs (default: "logs")
    """
    # Create log directory if it doesn't exist
    Path(log_dir).mkdir(parents=True, exist_ok=True)
    
    if verbose:
        print(f"\n{'='*80}", flush=True)
        print(f"Solving Task: {task_id}", flush=True)
        print(f"{'='*80}", flush=True)
    
    try:
        # ====================================================================
        # STEP 1: Find Similar Programs by Execution (PARALLELIZED)
        # ====================================================================
        if verbose:
            print("\n🔍 Finding similar programs by execution...", flush=True)
        
        similar_programs = library.find_similar(
            train_examples=task['train'],
            top_k=5,
            min_similarity=0.1,
            n_workers=n_workers,
            timeout=timeout
        )
        
        if verbose:
            if similar_programs:
                print(f"   Found {len(similar_programs)} similar programs:", flush=True)
                for i, prog in enumerate(similar_programs[:3], 1):
                    print(f"   {i}. Task {prog['task_id']}: {prog['similarity']:.2f}", flush=True)
            else:
                print("   No similar programs found", flush=True)
        
        # ====================================================================
        # TEST LIBRARY PROGRAMS: Try existing solutions first
        # ====================================================================
        best_library_score = 0.0
        best_library_program = None
        
        if similar_programs:
            best_match = similar_programs[0]
            best_library_score = best_match['similarity']
            best_library_program = best_match['program']
            if verbose:
                print(f"\n✓ Best library match: Task {best_match['task_id']} ({best_library_score:.2f})", flush=True)
            
            if best_library_score == 1.0:
                if verbose:
                    print(f"   → Perfect match found! Using library solution.", flush=True)
                return TaskResult(
                    task_id=task_id,
                    success=True,
                    score=1.0,
                    program=best_library_program
                )
                
        # ====================================================================
        # PHASE 1: Pattern Discovery (Natural Language)
        # ====================================================================
        if verbose:
            print("\n📝 Phase 1: Pattern Discovery...", flush=True)
        
        # Pass the raw similar_programs list - prompter will format it
        phase1_prompt = prompter.build_phase1_prompt(task, similar_programs)
        
        phase1_output = vlm_client_phase1.query(
            phase1_prompt,
            system_prompt=""""You are an expert at analyzing ARC puzzles and discovering transformation patterns.

Remember: Your first hypothesis is sticky and excessively convincing to you.
Combat this by evolving your hypothesis and actively seeking evidence against your initial guess to avoid halluciantion.
"""
        )
        
        # LOG PHASE 1 OUTPUT
        phase1_log_path = os.path.join(log_dir, f"{task_id}_phase1_output.txt")
        with open(phase1_log_path, 'w', encoding='utf-8') as f:
            f.write(f"Task ID: {task_id}\n")
            f.write("="*80 + "\n")
            f.write("PHASE 1: PATTERN DISCOVERY OUTPUT\n")
            f.write("="*80 + "\n\n")
            f.write(phase1_output)
        
        if verbose:
            print(f"   ✓ Phase 1 complete ({len(phase1_output)} chars)", flush=True)
            print(f"   📄 Logged to: {phase1_log_path}", flush=True)
        
        # ====================================================================
        # PHASE 2: Code Generation
        # ====================================================================
        if verbose:
            print("\n⚙️  Phase 2: Code Generation...", flush=True)
        
        # Pass the raw similar_programs list - prompter will format it
        phase2_prompt = prompter.build_phase2_prompt(task, 
            phase1_output,
            similar_programs
        )
        
        phase2_output = vlm_client_phase2.query(
            phase2_prompt,
            system_prompt="You are an expert at generating code using the given DSL primitives to solve ARC puzzles. You are provided with a natural language description of the pattern to implement, as well as training examples and some similar programs you might find useful as reference. Generate a Python function `def solve(I):` that implements the described transformation using ONLY the provided DSL primitives. Ensure your code is syntactically correct and follows best practices."
        )
        
        if verbose:
            print(f"   ✓ Phase 2 complete ({len(phase2_output)} chars)", flush=True)
        
        # ====================================================================
        # EXTRACT AND TEST GENERATED CODE
        # ====================================================================
        if verbose:
            print("\n🧪 Testing generated program...", flush=True)
        
        generated_code = extract_code_from_response(phase2_output)
        
        if not generated_code:
            if verbose:
                print("   ✗ Failed to extract code", flush=True)
            
            if best_library_program and best_library_score > 0.5:
                if verbose:
                    print(f"   → Falling back to library (score: {best_library_score:.2f})", flush=True)
                return TaskResult(
                    task_id=task_id,
                    success=False,
                    score=best_library_score,
                    program=best_library_program,
                    phase1_output=phase1_output,
                    error="Code extraction failed, using library fallback"
                )
            
            return TaskResult(
                task_id=task_id,
                success=False,
                score=0.0,
                phase1_output=phase1_output,
                error="Failed to extract code from response"
            )
        
        score, results = test_program(generated_code, task)
        
        if verbose:
            print(f"   Generated score: {score:.2f}", flush=True)
        
        # LOG PHASE 2 OUTPUT WITH TEST RESULTS
        phase2_log_path = os.path.join(log_dir, f"{task_id}_phase2_results.txt")
        with open(phase2_log_path, 'w', encoding='utf-8') as f:
            f.write(f"Task ID: {task_id}\n")
            f.write("="*80 + "\n")
            f.write("PHASE 2: CODE GENERATION & TEST RESULTS\n")
            f.write("="*80 + "\n\n")
            
            f.write("GENERATED CODE:\n")
            f.write("-"*80 + "\n")
            f.write(generated_code + "\n")
            f.write("-"*80 + "\n\n")
            
            f.write(f"SCORE: {score:.2f}\n\n")
            
            f.write("TEST RESULTS:\n")
            f.write("-"*80 + "\n")
            for i, (expected, actual, passed) in enumerate(results, 1):
                f.write(f"\nExample {i}: {'✓ PASS' if passed else '✗ FAIL'}\n")
                f.write(f"Expected Output:\n")
                if expected:
                    f.write(f"{json.dumps([list(row) for row in expected], indent=2)}\n")
                else:
                    f.write("None\n")
                f.write(f"Actual Output:\n")
                if actual:
                    f.write(f"{json.dumps([list(row) for row in actual], indent=2)}\n")
                else:
                    f.write("None (execution failed)\n")
                f.write("-"*40 + "\n")
        
        if verbose:
            print(f"   📄 Logged to: {phase2_log_path}", flush=True)
        
        # ====================================================================
        # DECIDE FINAL PROGRAM
        # ====================================================================
        success = score == 1.0
        final_program = generated_code
        final_score = score
        
        if not success and best_library_score > score:
            if verbose:
                print(f"   → Library program better ({best_library_score:.2f} > {score:.2f})", flush=True)
            final_program = best_library_program
            final_score = best_library_score
        
        # ====================================================================
        # SAVE TO LIBRARY IF SUCCESSFUL
        # ====================================================================
        if success:
            namespace = globals().copy()
            exec(final_program, namespace)
            if 'solve' in namespace:
                library.add(task_id, final_program)
                if verbose:
                    print(f"   ✓ Added to library", flush=True)
        
        return TaskResult(
            task_id=task_id,
            success=success,
            score=final_score,
            program=final_program,
            phase1_output=phase1_output
        )
        
    except Exception as e:
        if verbose:
            print(f"   ✗ Error: {e}", flush=True)
        
        return TaskResult(
            task_id=task_id,
            success=False,
            score=0.0,
            error=str(e)
        )


def process_directory(
    data_dir: str,
    vlm_client_phase1: BaseVLMClient,
    vlm_client_phase2: BaseVLMClient,
    prompter: VLMPrompter,
    library: ProgramLibrary,
    verbose: bool = True,
    n_workers: int = None,
    timeout: int = 2,
    evaluate_on_n_pb: int = -1,
) -> List[TaskResult]:
    """
    Process all task files in a directory.
    
    Args:
        data_dir: Directory containing task JSON files
        vlm_client: VLM client for queries
        prompter: Prompt builder
        library: Program library
        verbose: Print progress
        n_workers: Number of parallel workers for library search (None = auto)
        timeout: Timeout per program execution in seconds
    """
    data_path = Path(data_dir)
    
    if not data_path.exists():
        print(f"Error: Directory not found: {data_dir}", flush=True)
        return []
    
    json_files = sorted(data_path.glob('*.json'))
    
    if not json_files:
        print(f"No JSON files found in {data_dir}", flush=True)
        return []
    
    print(f"\nFound {len(json_files)} tasks in {data_dir}\n", flush=True)
    if evaluate_on_n_pb != -1:
        json_files = json_files[:evaluate_on_n_pb]
        print (f"Evaluating on first {evaluate_on_n_pb} tasks\n", flush=True)
    results = []
    successful = 0
    total_score = 0.0
    
    for i, task_file in enumerate(json_files, 1):
        task_id = task_file.stem
        
        try:
            with open(task_file, 'r') as f:
                task = json.load(f)
            
            result = solve_task(
                task=task,
                task_id=task_id,
                vlm_client_phase1=vlm_client_phase1,
                vlm_client_phase2=vlm_client_phase2,
                prompter=prompter,
                library=library,
                verbose=verbose,
                n_workers=n_workers,
                timeout=timeout,
                log_dir="logs_images"#TODO change log dir
            )
            
            results.append(result)
            
            if result.success:
                successful += 1
            
            total_score += result.score
            
            status = "✓" if result.success else "✗"
            print(f"{status} [{i}/{len(json_files)}] {task_id}: {result.score:.2f}", flush=True)
            
        except json.JSONDecodeError as e:
            print(f"✗ [{i}/{len(json_files)}] {task_id}: Invalid JSON - {e}", flush=True)
        except Exception as e:
            print(f"✗ [{i}/{len(json_files)}] {task_id}: {e}", flush=True)
    
    # Summary
    print(f"\n{'='*80}", flush=True)
    print(f"SUMMARY", flush=True)
    print(f"{'='*80}", flush=True)
    print(f"Total tasks: {len(json_files)}", flush=True)
    print(f"Successful: {successful}/{len(json_files)} ({100*successful/len(json_files):.1f}%)", flush=True)
    print(f"Average score: {total_score/len(json_files):.2f}", flush=True)
    print(f"{'='*80}\n", flush=True)
    
    return results


def save_results(results: List[TaskResult], output_dir: str = 'results') -> None:
    """Save results to JSON and CSV files."""
    import csv
    
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # JSON
    json_file = output_path / 'results.json'
    with open(json_file, 'w') as f:
        json_data = [
            {
                'task_id': r.task_id,
                'success': r.success,
                'score': r.score,
                'error': r.error,
                'program': r.program,
            }
            for r in results
        ]
        json.dump(json_data, f, indent=2)
    print(f"Saved detailed results to {json_file}", flush=True)
    
    # CSV
    csv_file = output_path / 'summary.csv'
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['task_id', 'success', 'score', 'error'])
        for r in results:
            writer.writerow([r.task_id, r.success, f'{r.score:.2f}', r.error or ''])
    print(f"Saved summary to {csv_file}", flush=True)


def main():
    from dotenv import load_dotenv
    """Main entry point"""
    # print("Initializing components...", flush=True)
    load_dotenv()
    PROVIDER = "grok"
    if PROVIDER == "grok":
        api_key = os.getenv('GROK_API_KEY')
        api_base = "https://api.x.ai/v1"
        model = "grok-4-fast"
    elif PROVIDER == "qwen":
        api_key = None
        api_base = "http://localhost:8000/v1"
        model = "Qwen/Qwen2.5-7B-Instruct"
    elif PROVIDER == "gemini":
        api_key = os.getenv('GEMINI_API_KEY')
        api_base = "https://generativelanguage.googleapis.com/v1beta/models/"
        model = "gemini-2.5-pro"
        
    vlm_config_phase1 = VLMConfig(
        api_key=api_key,
        model=model,
        api_base=api_base,
        max_tokens=16384,  # Longer for analysis
        save_prompts=False,
        prompt_log_dir="prompts_testing"
    )
    vlm_config_phase2 = VLMConfig(
        api_key=api_key,
        model=model,
        api_base=api_base,
        max_tokens=8192   # Shorter for code gen
    )
    
    vlm_client_phase1 = create_client(PROVIDER, config=vlm_config_phase1)
    # print("VLM client created", flush=True)
    
    vlm_client_phase2 = create_client(PROVIDER, config=vlm_config_phase2)
    prompter = VLMPrompter()
    # print("Prompter created", flush=True)
    
    library = ProgramLibrary()  # Auto-loads from solvers.py
    # print("Loaded library...", flush=True)
    #sanity check
    print(f"Loaded {len(library)} programs from library", flush=True)
    if len(library) > 0:
        print(f"First program: {library.programs[0]['task_id']}", flush=True)
    
    # Configure parallelization
    results = process_directory(
        data_dir='data_v1/eval_size_10',
        vlm_client_phase1=vlm_client_phase1,
        vlm_client_phase2=vlm_client_phase2,
        prompter=prompter,
        library=library,
        verbose=True,
        n_workers=None,  # Auto-detect CPUs (recommended)
        timeout=2        # 2 second timeout per program
    )
    
    save_results(results, output_dir='results/images')#TODO change output dir


# if __name__ == "__main__":
#     # print("Starting main...", flush=True)
#     sys.stdout.flush()
#     main()

In [3]:
from dotenv import load_dotenv
load_dotenv()
api_key = None
api_base = "http://localhost:8000/v1"
model = "Qwen/Qwen2.5-7B-Instruct"
PROVIDER = "qwen"
vlm_config_phase1 = VLMConfig(
    api_key=api_key,
    model=model,
    api_base=api_base,
    max_tokens=16384,  # Longer for analysis
    save_prompts=False,
    prompt_log_dir="prompts_testing"
)
vlm_config_phase2 = VLMConfig(
    api_key=api_key,
    model=model,
    api_base=api_base,
    max_tokens=8192   # Shorter for code gen
)
vlm_client_phase1 = create_client(PROVIDER, config=vlm_config_phase1)
# print("VLM client created", flush=True)

vlm_client_phase2 = create_client(PROVIDER, config=vlm_config_phase2)
prompter = VLMPrompter(use_vision=False)

library = ProgramLibrary()  # Auto-loads from solvers.py



In [4]:
print(f"Loaded {len(library)} programs from library", flush=True)
if len(library) > 0:
    print(f"First program: {library.programs[0]['task_id']}", flush=True)


Loaded 400 programs from library
First program: 007bbfb7


In [5]:
data_path = "/home/flowers/work/llms_ftw/tasks/evaluation/"
data_path = Path(data_path)
json_files = sorted(data_path.glob('*.json'))
json_files[:10]

[PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/0934a4d8.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/135a2760.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/136b0064.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/13e47133.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/142ca369.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/16b78196.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/16de56c4.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/1818057f.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/195c6913.json'),
 PosixPath('/home/flowers/work/llms_ftw/tasks/evaluation/1ae2feb7.json')]

In [6]:
data_dir = "/home/flowers/work/llms_ftw/tasks/evaluation/"
verbose = True
n_workers = None
timeout = 2
evaluate_on_n_pb = 1


data_path = Path(data_dir)


json_files = sorted(data_path.glob('*.json'))


print(f"\nFound {len(json_files)} tasks in {data_dir}\n", flush=True)
if evaluate_on_n_pb != -1:
    json_files = json_files[:evaluate_on_n_pb]
    print (f"Evaluating on first {evaluate_on_n_pb} tasks\n", flush=True)
results = []
successful = 0
total_score = 0.0

i=1
task_file = json_files[0]
task_id = task_file.stem
with open(task_file, 'r') as f:
    task = json.load(f)

task=task
task_id=task_id
# vlm_client_phase1=vlm_client_phase1
# vlm_client_phase2=vlm_client_phase2
# prompter=prompter
# library=library
verbose=verbose
n_workers=n_workers
timeout=timeout
log_dir="logs"#TODO change log dir

# result = solve_task(
# task=task,
# task_id=task_id,
# vlm_client_phase1=vlm_client_phase1,
# vlm_client_phase2=vlm_client_phase2,
# prompter=prompter,
# library=library,
# verbose=verbose,
# n_workers=n_workers,
# timeout=timeout,
# log_dir="logs_images"#TODO change log dir
# )



Found 120 tasks in /home/flowers/work/llms_ftw/tasks/evaluation/

Evaluating on first 1 tasks



In [7]:
similar_programs = library.find_similar(
    train_examples=task['train'],
    top_k=5,
    min_similarity=0.1,
    n_workers=n_workers,
    timeout=timeout
)



=== DEBUG find_similar ===
Number of train examples: 4
Number of programs in library: 400
min_similarity threshold: 0.1
top_k: 5
Using 28 parallel workers (timeout: 2s per program)
Progress: 50/400 programs (178.6/s, 0.3s elapsed)
Progress: 100/400 programs (217.6/s, 0.5s elapsed)
Progress: 150/400 programs (237.6/s, 0.6s elapsed)
Progress: 200/400 programs (263.8/s, 0.8s elapsed)
Progress: 250/400 programs (264.5/s, 0.9s elapsed)
Progress: 300/400 programs (270.0/s, 1.1s elapsed)
Progress: 350/400 programs (244.2/s, 1.4s elapsed)
Progress: 400/400 programs (26.0/s, 15.4s elapsed)

Evaluation complete: 400/400 programs in 15.5s

=== Execution Statistics ===
Programs with errors: 31/400
Max similarity achieved: 0.110
Average similarity: 0.007

Sample errors from first 3 programs:
  228f6490: StopIteration: 
  272f95fa: StopIteration: 
  3de23699: StopIteration: 

Top 5 programs by similarity:
  1cf80156: 0.110 (errors: 0/4)
  ce4f8723: 0.103 (errors: 0/4)
  f25fbde4: 0.101 (errors: 0/4

In [10]:
best_library_score = 0.0
best_library_program = None

if similar_programs:
    best_match = similar_programs[0]
    best_library_score = best_match['similarity']
    best_library_program = best_match['program']
    if verbose:
        print(f"\n✓ Best library match: Task {best_match['task_id']} ({best_library_score:.2f})", flush=True)
    



✓ Best library match: Task 1cf80156 (0.11)


In [11]:
phase1_prompt = prompter.build_phase1_prompt(task, similar_programs)
phase1_output = vlm_client_phase1.query(
    phase1_prompt,
    system_prompt=""""You are an expert at analyzing ARC puzzles and discovering transformation patterns.

Remember: Your first hypothesis is sticky and excessively convincing to you.
Combat this by evolving your hypothesis and actively seeking evidence against your initial guess to avoid halluciantion.
"""
)


In [14]:
phase2_prompt = prompter.build_phase2_prompt(task, 
    phase1_output,
    similar_programs
)

phase2_output = vlm_client_phase2.query(
    phase2_prompt,
    system_prompt="You are an expert at generating code using the given DSL primitives to solve ARC puzzles. You are provided with a natural language description of the pattern to implement, as well as training examples and some similar programs you might find useful as reference. Generate a Python function `def solve(I):` that implements the described transformation using ONLY the provided DSL primitives. Ensure your code is syntactically correct and follows best practices."
)

if verbose:
    print(f"   ✓ Phase 2 complete ({len(phase2_output)} chars)", flush=True)


   ✓ Phase 2 complete (1337 chars)


In [19]:
generated_code = extract_code_from_response(phase2_output)
score, results = test_program(generated_code, task)
print(f"   Generated score: {score:.2f}", flush=True)


   Generated score: 0.13


In [ ]:
python -m sglang.launch_server --model-path /home/flowers/work/hf/Qwen3-4B-Instruct-2507 --tp 1 --port 8000 --mem-fraction-static 0.9 --random-seed 42 --host 0.0.0.0 --log-level info --trust-remote-code --quantization fp8 --context-length 32000 


In [ ]:
results = process_directory(
    data_dir='data_v1/eval_size_10',
    vlm_client_phase1=vlm_client_phase1,
    vlm_client_phase2=vlm_client_phase2,
    prompter=prompter,
    library=library,
    verbose=True,
    n_workers=None,  # Auto-detect CPUs (recommended)
    timeout=2        # 2 second timeout per program
)

# save_results(results, output_dir='results/images')#TODO change output dir
